In [1]:
import re
import pandas as pd
import numpy as np
import time
import sys

In [2]:
train_df=pd.read_csv('train_E6oV3lV.csv')
        #print(train_df.head())
        #id,label,tweet
        
# removing @user pattern
def remove_pattern(input_txt,pattern):
    r = re.findall(pattern,input_txt)
    for i in r:
        input_txt=re.sub(i,'',input_txt)
    return input_txt
train_df['tidy_tweet']=np.vectorize(remove_pattern)(train_df['tweet'],'@[\w]*')

# remove punctuation, special chr and numbers
train_df['tidy_tweet'] = train_df['tidy_tweet'].str.replace("[^a-zA-Z]", " ")

# Remove short words
train_df['tidy_tweet'] = train_df['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

#arrays
tweets=np.array(train_df['tidy_tweet'])
labels=np.array(train_df['label'])
train_df.head()

,id,label,tweet,tidy_tweet
0,1,0,@user when a father is dysfunctional and is s...,when father dysfunctional selfish drags kids i...
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks lyft credit cause they offer wheelchair...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,model love take with time
4,5,0,factsguide: society now #motivation,factsguide society motivation


##  basic Neural Network 

In [5]:
from collections import Counter
import numpy as np 
import pandas as pd 
class NeuralNetwork():
    def __init__(self, tweets, labels,hidden_nodes=10, learning_rate=0.1,):
        self.tweets = tweets
        self.labels = labels
        self.learning_rate = learning_rate
        np.random.seed(1)
        self.preprocessing_data(tweets, labels)
        self.init_network(len(self.vocab), hidden_nodes, 1,learning_rate)
        print('initialized NeuralNetwork class')

    def preprocessing_data(self, tweets, labels):
        vocab = set()
        for tweet in tweets:
            for word in tweet.split(' '):
                vocab.add(word)
        self.vocab=list(vocab)
        
        #indexing total vocabulary words from the dataset 
        self.indexed_vocab = {}
        for i, word in enumerate(self.vocab):
            self.indexed_vocab[word] = i
        self.vocab_size=len(self.vocab)
        
        #initializing  network with weights and layer zero
    def init_network(self, input_nodes, hidden_nodes, output_nodes,learning_rate):
        self.input_nodes = input_nodes
        self.hidden_nodes=hidden_nodes
        self.output_nodes=output_nodes
        self.learning_rate=learning_rate
        
        #here input node is processed with one node at a time thats why the vector is 1,input_nodes
        self.layer_0 = np.zeros((1, input_nodes))
        
        #Weights nodes
        self.weigth_0_1=np.zeros((self.input_nodes,self.hidden_nodes))
        self.weigth_1_2=np.random.normal(0.0, self.output_nodes**-0.5,(self.hidden_nodes, self.output_nodes))
        print('initialized neural net')

    # each tweet is injected into the train loop with preprocessing
    def  tweet_injection(self,tweet):
        self.layer_0 *=0
        for word in tweet.split(' '):
            if(word in self.indexed_vocab.keys()):
                self.layer_0[0][self.indexed_vocab[word]]=1
        
    
    def sigmoud(self,x):
        return 1/(1+np.exp(-x))
    
    def sigmoud_derivative_output(self,output):
        return output*(1-output)
    
    def train(self,training_tweets,trainging_labels):
        
        assert(len(training_tweets)==len(trainging_labels))
        correct_so_far = 0
        #  printing time statistics
        start = time.time()
        
        #Training loop starts here 
        for i in range(len(training_tweets)):
            
            tweet=training_tweets[i]
            label=trainging_labels[i]

            #Feed Forward
            #layer_0 is input nodes layer
            #layer_1 is hidden nodes layer
            #layer_2 is out put nodes layer
                #input layer
            self.tweet_injection(tweet)
                #hidden layer
            layer_1=self.layer_0.dot(self.weigth_0_1)
                #output layer
            layer_2=self.sigmoud(layer_1.dot(self.weigth_1_2))
            


            #Backward Propogation
                #output error
            error_layer2= layer_2-label
                #delta layer_2
            delta_layer2=error_layer2*self.sigmoud_derivative_output(layer_2)
                #hidden error
            
                #delta layer_1
            delta_layer1=delta_layer2.dot(self.weigth_1_2.T)


                #weights update
            self.weigth_0_1-=self.layer_0.T.dot(delta_layer1)*self.learning_rate
            self.weigth_1_2-=layer_1.T.dot(delta_layer2)*self.learning_rate

            #keep track of correct predictions
            if(layer_2 >= 0.5 and label == 1):
                correct_so_far += 1
            elif(layer_2 < 0.5 and label == 0):
                correct_so_far += 1
                
            #print acuuracy speed and training sizes
            elapsed_time = float(time.time() - start)
            reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0
            
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(training_tweets)))[:4] \
                             + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5] \
                             + " #Correct:" + str(correct_so_far) + " #Trained:" + str(i+1) \
                             + " Training Accuracy:" + str(correct_so_far * 100 / float(i+1))[:4] + "%")
            if(i % 2500 == 0):
                print("")


    def test(self, testing_tweets, testing_labels):
        """
        Attempts to predict the labels for the given testing_reviews,
        and uses the test_labels to calculate the accuracy of those predictions.
        """
        
        # keep track of how many correct predictions we make
        correct = 0

        # we'll time how many predictions per second we make
        start = time.time()

        # Loop through each of the given reviews and call run to predict
        # its label. 
        for i in range(len(testing_tweets)):
            pred = self.run(testing_tweets[i])
            if(pred == testing_labels[i]):
                correct += 1
            
            # For debug purposes, print out our prediction accuracy and speed 
            # throughout the prediction process. 

            elapsed_time = float(time.time() - start)
            reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0
            
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(testing_tweets)))[:4] \
                             + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5] \
                             + " #Correct:" + str(correct) + " #Tested:" + str(i+1) \
                             + " Testing Accuracy:" + str(correct * 100 / float(i+1))[:4] + "%")
    
    def run(self, tweets):
        """
        Returns a POSITIVE or NEGATIVE prediction for the given review.
        """
        # Run a forward pass through the network, like in the "train" function.
        
        # Input Layer
        self.tweet_injection(tweets.lower())

        # Hidden layer
        layer_1 = self.layer_0.dot(self.weigth_0_1)

        # Output layer
        layer_2 = self.sigmoud(layer_1.dot(self.weigth_1_2))
        
        # Return POSITIVE for values above greater-than-or-equal-to 0.5 in the output layer;
        # return NEGATIVE for other values
        if(layer_2[0] >= 0.5):
            return 1
        else:
            return 0

In [6]:
mlp = NeuralNetwork(tweets[:-1000],labels[:-1000], learning_rate=0.1)

preprocessed data
initialized neural net
initialized NeuralNetwork class


In [7]:
mlp.train(tweets[:-1000],labels[:-1000])

Progress:0.0% Speed(reviews/sec):0.0 #Correct:0 #Trained:1 Training Accuracy:0.0%
Progress:8.07% Speed(reviews/sec):142.9 #Correct:2276 #Trained:2501 Training Accuracy:91.0%
Progress:16.1% Speed(reviews/sec):143.0 #Correct:4620 #Trained:5001 Training Accuracy:92.3%
Progress:24.2% Speed(reviews/sec):115.3 #Correct:6954 #Trained:7501 Training Accuracy:92.7%
Progress:32.2% Speed(reviews/sec):99.84 #Correct:9326 #Trained:10001 Training Accuracy:93.2%
Progress:40.3% Speed(reviews/sec):93.05 #Correct:11680 #Trained:12501 Training Accuracy:93.4%
Progress:48.4% Speed(reviews/sec):88.94 #Correct:14033 #Trained:15001 Training Accuracy:93.5%
Progress:56.5% Speed(reviews/sec):86.13 #Correct:16406 #Trained:17501 Training Accuracy:93.7%
Progress:64.5% Speed(reviews/sec):84.24 #Correct:18784 #Trained:20001 Training Accuracy:93.9%
Progress:72.6% Speed(reviews/sec):82.62 #Correct:21146 #Trained:22501 Training Accuracy:93.9%
Progress:80.7% Speed(reviews/sec):81.56 #Correct:23522 #Trained:25001 Training 

In [8]:
mlp.test(tweets[-500:],labels[-500:])

Progress:20.2% Speed(reviews/sec):505.1 #Correct:100 #Tested:102 Testing Accuracy:98.0%

Progress:40.6% Speed(reviews/sec):497.6 #Correct:197 #Tested:204 Testing Accuracy:96.5%

Progress:99.8% Speed(reviews/sec):493.7 #Correct:472 #Tested:500 Testing Accuracy:94.4%